In [3]:
# !pip install optuna -q
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["figure.dpi"] = 100

# Training the final model with all training data

In [7]:
file_path = 'training_data.csv'
data = pd.read_csv(file_path)

data['increase_stock_binary'] = data['increase_stock'].map(lambda x: 0 if x == 'low_bike_demand' else 1)
data = data.drop(['snow', 'increase_stock'], axis=1)
# pick out the labels
y = data['increase_stock_binary'].to_numpy().astype(np.float32)
y = torch.tensor(y).unsqueeze(-1)
data = data.drop(['increase_stock_binary'], axis=1)


# Separating the numerical and categorical data to handle them separately
onehot = True
if onehot:
    cat_data = pd.DataFrame({key:data[key] for key in data.keys() if data[key].dtype == int})
    num_data = pd.DataFrame({key:data[key] for key in data.keys() if data[key].dtype == float})
    
    # performing onehot encoding on the categorical data
    cat_data = torch.from_numpy(cat_data.to_numpy())
    cat_onehot = torch.cat([F.one_hot(x, num_classes=24) for x in cat_data]).view(cat_data.shape[0], -1)
    
    # constructing the complete input dataset
    data = np.concatenate([num_data.to_numpy(), cat_onehot.numpy()], axis=-1).astype(np.float32)

# scaling
X = MinMaxScaler().fit_transform(data) 
X = torch.Tensor(X)

#X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, shuffle=True, random_state=0)
X_train = X
y_train = y

In [8]:
class DNN(nn.Module):
    def __init__(self, hidden_size, input_size, layers=2, seed=0, submodule=False):
        super().__init__()
        torch.manual_seed(seed)
        
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.submodule = submodule
        
        self.activation = nn.ReLU()
        self.input_layer = nn.Linear(input_size, hidden_size)
        self.batch_norm_input = nn.BatchNorm1d(hidden_size)  # BatchNorm for input layer
        self.linear_layers = nn.ModuleList()
        self.batch_norm_layers = nn.ModuleList()  # BatchNorm for other layers

        for _ in range(layers):
            self.linear_layers.append(nn.Linear(hidden_size, hidden_size))
            self.batch_norm_layers.append(nn.BatchNorm1d(hidden_size))

        self.output_layer = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = self.batch_norm_input(x)
        x = self.activation(x)

        for layer, batch_norm in zip(self.linear_layers, self.batch_norm_layers):
            residual = x
            x = layer(x)
            x = batch_norm(x)  
            x = self.activation(x + residual)
        
        if not self.submodule:
            x = self.output_layer(x)
            x = torch.sigmoid(x) 
        return x

In [51]:
epochs = 100 
loss_fn = nn.BCELoss()
n_ensembles = 10 # checking the performance for different weight initialization
kf = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
optimal_models = []

for n in range(n_ensembles):
    for cv, (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
        model = DNN(200, X.shape[-1], 2, seed=n)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1)
        
        train_loss_history = []
        
        for epoch in tqdm(range(epochs)):
            optimizer.zero_grad()
            model.train()
            y_pred = model(X_train[train_index])
            loss = loss_fn(y_pred, y_train[train_index])
            loss.backward()
            optimizer.step()
            train_loss_history.append(loss.item())
            
            if loss.item() <= min(train_loss_history):
                optimal_weights = model.state_dict()
                
        optimal_models.append(optimal_weights)  

100%|██████████| 100/100 [00:01<00:00, 54.24it/s]


# Testing

In [54]:
file_path = 'test_data.csv'
data = pd.read_csv(file_path)
data = data.drop(['snow'], axis=1)


# Separating the numerical and categorical data to handle them separately
onehot = True
if onehot:
    cat_data = pd.DataFrame({key:data[key] for key in data.keys() if data[key].dtype == int})
    num_data = pd.DataFrame({key:data[key] for key in data.keys() if data[key].dtype == float})
    
    # performing onehot encoding on the categorical data
    cat_data = torch.from_numpy(cat_data.to_numpy())
    cat_onehot = torch.cat([F.one_hot(x, num_classes=24) for x in cat_data]).view(cat_data.shape[0], -1)
    
    # constructing the complete input dataset
    data = np.concatenate([num_data.to_numpy(), cat_onehot.numpy()], axis=-1).astype(np.float32)

# scaling
X = MinMaxScaler().fit_transform(data) 
X_test = torch.Tensor(X)

In [55]:
def DeepNeuralNetworkEnsemble(optimal_models, test_data):
    DNN_ensemble = []
    for weights in optimal_models:
        model = DNN(200, X.shape[-1], 2)
        model.load_state_dict(weights)
        y_pred = model(test_data)
        DNN_ensemble.append(y_pred.detach().numpy())
    return torch.Tensor(DNN_ensemble).mean(0).round()
    
prediction = DeepNeuralNetworkEnsemble(optimal_models, X_test)

In [59]:
py.savetxt("DNN test prediction .csv", prediction.detach().numpy().flatten().astype(int), delimiter=",")

array([0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,